# We Rate Dogs Wrangle Report #
–––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

## Introduction ##

This report covers the data wrangling efforts to gather, assess and clean data from the 'We Rate Dogs' twitter page and associated dog breed predictions data provided by Udacity. 

## Data Gathering ##

For this project, data was initially gathered from 3 sources:  
1. Udacity provided WeRateDogs Twitter archive data found in the "twitter-archive-enhanced.csv" file.
2. Image predictions for the twitter archive data, predicting what breed of dog (or other object, animal, etc.) is in each tweet according to Udacity's neural network. The image predictions were provided in a file, 'image_predictions.tsv' hosted on Udacity's servers that was downloaded programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv
3. Finally, each tweet's retweet count and favorite ("like") count was queried from the Twitter's API using the tweet IDs in the WeRateDogs Twitter archive.  More specifically, I queried the Twitter API for each tweet's JSON data using Python's Tweepy library and stored each tweet's entire set of JSON data in a file called tweet_json.txt file.  The tweet_json.txt file was then read line by line into a pandas dataframe for analysis and eventual merging with the existing WeRateDogs twitter archive.

## Data Assessing ##

After the data was gathered, I assessed the data for data quality and data tidiness issues as per the following requirements:  

**Data Quality Issues:** Issues with content in which data is low quality otherwise known as 'dirty' data as per the following data quality requirements:
1. Completeness: do we have all of the records that we should? Do we have missing records or not? Are there specific rows, columns, or cells missing?
2. Validity: we have the records, but they're not valid, i.e., they don't conform to a defined schema. A schema is a defined set of rules for data. These rules can be real-world constraints (e.g. negative height is impossible) and table-specific constraints (e.g. unique key constraints in tables).
3. Accuracy: inaccurate data is wrong data that is valid. It adheres to the defined schema, but it is still incorrect. Example: a patient's weight that is 5 lbs too heavy because the scale was faulty.
4. Consistency: inconsistent data is both valid and accurate, but there are multiple correct ways of referring to the same thing. Consistency, i.e., a standard format, in columns that represent the same data across tables and/or within tables is desired.

**Data Tidiness Issues:** Issues with structure that prevent easy analysis in which  data is 'untidy', otherwise known as 'messy' data as per the following Tidy data requirements:
1. Each variable forms a column.
2. Each observation forms a row.
3. Each type of observational unit forms a table.

To perform my assessment I used both visual and programmatic assessment, with the latter being the majority of my efforts given it is much more efficient.  Specifically, for visual assessment: I scrolled through the data in Jupyter Notebooks and for programmatic assessment: I used code to view specific portions and summaries of the data (e.g. pandas' head, tail, info, duplicated, isna etc. methods).

## Data Cleaning ##

As a result of my data quality and data tidiness assessments, I identified the following issues for which the associated data cleaning steps I undertook are also described below.  In total, across the 3 data sources there were 12 data quality issues and 5 data tidiness issues I identified and subsequently cleaned -- I guess I was feeling ambitious and wanted to ensure good data for my subsequent analysis (see accompanying act_report).

### Twitter Archive Enhanced ###
**Data Quality Issues:**
1. For the dog 'stage' column, many entries were populated with 'None' when the information was missing.  To remedy as part of data cleaning, I replaceed these instances with NaN instead to better allow analysis via Pandas provided methods like 'isna' etc.
2. Similarly, for the dog 'name' column, there were 745 instances of 'None', which I replaced with NaN.
3. The columns 'timestamp' and 'retweeted_status_timestamp' were of string format which I cleaned by converting the columns to date_time dataype via Pandas 'to_datetime' function.
4. There were 181 retweets whereas this project only called for anlaysis of original tweets.  To clean this, I removed these instances via a combination of drop and index methods.
5. There were a handful of extremely high ratings (>20) in the rating_numerator column which appeared to have arisen due to inconsistent tweet text string patterns that caused the extraction to grab the wrong numbers.  As these instances were only a handful (only 4 after the previous 4 data cleansing steps), I simply manually corrected them using loc method and assignment.
6. There were similarly a handful of incorrect rating_denominator values (i.e. not equal to 10) resulting from the same cause as the prior issue. Again, after the prior clean up steps, there were only about 8 such instances and so they were cleaned manually using loc method and assignment.
7. There were roughly 70 instances of reply tweets to original tweets without new dog postings.  To clean, I removed these instaces via a combination of drop and index methods.
8. There were 2 instances of rating_numerators of zero (0).  Upon further analysis those tweets were not for actual dog postings (tweeted complaints about plagiarism and a poor view from the empire state building) and so these instances were dropped.
9. The tweet 'source' column contained the full html tag information.  To clean for simplicity and clarity during analysis, I applied str.replace method.

**Data Tidiness Issues**
1. There were multiple columns for the one variable dog stage (each column representing one possible dog stage).  To clean, I collapsed these separate columns into a single new column 'stage' and assigned datatype of category to the new column.  To accomplish this, I first replaced missing values with ' ', then concatenated the values from each of the existing columns using the 'cat' method, finally in instances where no dog stage was identified I replaced with NaN and lastly dropped the old columns.

2.  There were multiple retweet related columns (e.g. retweeted_status_id, retweeted_user_id etc.) which didn't really belong in this table as they represented a separate observation.  For my analysis purposes, I didn't need these columns and so for clarity I drop the retweet related columns.

3. I dropped the 'in_reply' columns as there were very few such rows populated and they weren't relevant for my subsequent analysis.  Prior to dropping, I copied this data to a separate smaller data frame in case someone would want to look at it separately for a different analysis effort.

### Image Predictions ###
**Data Quality**

1.  There were 66 instances of duplicate predictions (i.e. whereby both the original tweet and the retweeted image were used).  To clean, I dropped these instances using the drop_duplicates method.
2. There were less image predictions than recoreds in the WeRateDogs twitter archive.  I moved the extra archive records to a separate dataframe in case I might need them later and then dropped them from the main dataframe to simplify our subsequent analysis to only focus on those that had predictions provided.

**Data Tidiness Issues**
1. For analysis purposes, I merged the highest confidence dog breed predictions (i.e. p1) and associated confidence p1_conf for instances where it is in fact a dog p1_dog = True, with the twitter archive using the merge method.

### Assess: Twitter API Data ###
**Data Quality Issues**

1. To account for the fact that the twitter archive had a few records for which the twitter API encountered an error retrieving data (e.g. tweet no longer existed etc.), I performed an inner join type merge as part of my cleaning for data tidiness in the next step.  This removed these incomplete records (i.e. archive records without supplementary info about retweet and favorite counts).

**Data Tidiness Issues**

1. To simplify analysis, the Twitter API data was combined with the twitter archive into a single dataframe using the Pandas merge function.  Now each tweet record in the WeRateDogs twitter archive dataframe also has info about retweet and favorite count which is in line with data tidiness requirements that each row represent one observation -- each record contains the relevant metrics about a tweet.

Upon completion of the data wrangling efforts, the final data was saved in the file "twitter_archive_enhanced.csv".